#### import dependencies

In [1]:
import pandas as pd
import requests
import json
# Google developer API key
from config import gkey

#### import datasets: college data, lat/lng from city and state

In [4]:
college_df = pd.read_csv('../Niche_Scraping/top800_best_colleges_in_America_2022.csv').iloc[:, 1:]
college_df['city'] = college_df['city'].str.title()
college_df['city'] = college_df['city'].str.replace('Saint','St.')
college_df['state'] = college_df['state'].str.strip()
college_df['location'] = college_df['city']+', '+college_df['state']
college_df.head()

,name,rating,location,niche_grade,acceptance_rate (%),net_price ($),SAT_range,city,state,SAT_low,SAT_high,SAT_average
0,Massachusetts Institute of Technology,1,"Cambridge, MA",A+,7,20465.0,1510-1570,Cambridge,MA,1510.0,1570.0,1540.0
1,Harvard University,2,"Cambridge, MA",A+,5,18030.0,1460-1570,Cambridge,MA,1460.0,1570.0,1515.0
2,Stanford University,3,"Stanford, CA",A+,4,16779.0,1440-1570,Stanford,CA,1440.0,1570.0,1505.0
3,Yale University,4,"New Haven, CT",A+,6,18073.0,1460-1570,New Haven,CT,1460.0,1570.0,1515.0
4,Princeton University,5,"Princeton, NJ",A+,6,18712.0,1460-1570,Princeton,NJ,1460.0,1570.0,1515.0


### get locations of city & state

In [7]:
# Target cities list
cities = college_df['location'].unique().tolist()
type(cities)

list

In [8]:
len(cities)

599

In [9]:
#this step takes a little bit long
city_lat = []
city_lng = []

for city in cities:
    # Build the endpoint URL
    target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={city}&key={gkey}"
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()
    # Extract latitude and longitude
    city_lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
    city_lng.append(geo_data["results"][0]["geometry"]["location"]["lng"])

In [14]:
city_df = pd.DataFrame(list(zip(cities, city_lat,city_lng)), columns =['location', 'city_lat','city_lng'])
city_df.head()

,location,city_lat,city_lng
0,"Cambridge, MA",42.373616,-71.109733
1,"Stanford, CA",37.424106,-122.166076
2,"New Haven, CT",41.308274,-72.927883
3,"Princeton, NJ",40.357298,-74.667223
4,"Durham, NC",35.994033,-78.898619


In [16]:
#check if there are any missing values
city_df.isnull().values.any()

False

In [18]:
city_df.to_csv("city_lat_lng.csv",index=False)

### get locations of university

In [19]:
# Target cities list
colleges = college_df['name'].unique().tolist()
len(colleges)

800

In [22]:
#this step takes a little bit long
college_lat = []
college_lng = []

for college in colleges:
    # Build the endpoint URL
    target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={college}&key={gkey}"
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()
    # Extract latitude and longitude
    college_lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
    college_lng.append(geo_data["results"][0]["geometry"]["location"]["lng"])

In [23]:
len(college_lat)

800

In [24]:
len(college_lng)

800

In [25]:
college_location_df = pd.DataFrame(list(zip(colleges, college_lat,college_lng)), columns =['name', 'college_lat','college_lng'])
college_location_df.head()

,name,college_lat,college_lng
0,Massachusetts Institute of Technology,42.360091,-71.094160
1,Harvard University,42.377003,-71.116660
2,Stanford University,37.427475,-122.169719
3,Yale University,41.316324,-72.922343
4,Princeton University,40.343094,-74.655074


In [26]:
#check if there are any missing values
college_location_df.isnull().values.any()

False

In [27]:
college_location_df.to_csv("college_lat_lng.csv",index=False)